In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import seaborn as sns
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
sns.set_theme()


params = {'legend.fontsize': 18,
          'legend.handlelength': 2,
          'figure.figsize': (15, 8),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

### Set filepath to default.log in cell below. 
Leave the *r* in front of the qoutes

In [40]:
#####################-FILEPATH STRING-###################################
filepath = r"C:\Users\SpencerTeolis\TRX Systems\PRGM - US Army TRX DAPS 1.2 - Documents\Testing\2021-02-25\2020-02-25 - window power test\000C\default.log"
#########################################################################

filepath = filepath.replace(os.sep, '/')

with open(filepath, 'r') as file_object:
        line = file_object.readline()
        battmon = []
        power = []
        while line:
            if '[battmon] [info]' in line:
                i = line.find("Temp")
                values = [x[2:-1] if x[2:-1].isalpha() else float(x[2:-1])
                          for x in re.findall(r": .*?[,\n]", line[i:])]
                
                values.insert(0, pd.Timestamp(line[1:24]))
                battmon.append(values)
            elif '[power] [info]' in line:
                i = line.find("[info]") + 7
                try:
                    state = re.search(r"[:-].*", line[i:]).group(0)[2:]
                except AttributeError:
                    print(line[i:])
                if not power or power[-1][-1] != state:
                    power.append([pd.Timestamp(line[1:24]),state])
        
            line = file_object.readline()

### Set the state index strings if needed 
These are just meant to define starts and ends of logs and will give you a list of possible start and end points to choose from in when graphing.<br>
They should be set to states that appear in the log. <br>

In [50]:
################-START AND END INDEX STRINGS-############################
beginning_state_string = "monitorRunner::Start"
end_state_string = "StopSystem::End"
#########################################################################

powe_columns = ['Time', 'State']
batt_columns = ['Time', 'Temperature', 'Voltage', 'Cap', 'Current', 
                'Power', 'Soc', 'IsChg', 'IsExtPwr']
batt_df = pd.DataFrame.from_dict({c : d for c, d in zip(batt_columns, zip(*battmon))})
powe_df = pd.DataFrame.from_dict({c : d for c, d in zip(powe_columns, zip(*power))})

batt_df["Minutes"] = (batt_df["Time"] - batt_df["Time"].iloc[0]) 
batt_df["Minutes"] = batt_df["Minutes"].dt.total_seconds() / 60
batt_df["Power"] *= -1

powe_df["Minutes"] = (powe_df["Time"] - powe_df["Time"].iloc[0])
powe_df["Minutes"] = powe_df["Minutes"].dt.total_seconds() / 60

starts = powe_df.index[powe_df["State"] == beginning_state_string].tolist()
ends = powe_df.index[powe_df["State"] == end_state_string].tolist()
ends.append(len(powe_df) - 1)



In [51]:
from matplotlib import lines

def power_profile(batt_df, state_df, x_axis, y_axis, y_axis2, start_index, end_index, full_string):
    fig, ax = plt.subplots(figsize=(15, 8)) 
    twin_ax = True if y_axis2 else False
    
    start_time = powe_df["Time"].loc[start_index]
    end_time = powe_df["Time"].loc[end_index]
    
    state_df = state_df[state_df["Time"].between(start_time, end_time)].copy()
    batt_df = batt_df[batt_df["Time"].between(start_time, end_time)].copy()
    state_df["Minutes"] -= state_df["Minutes"].iloc[0]
    batt_df["Minutes"] -= batt_df["Minutes"].iloc[0]
    
    bbox = ax.get_window_extent()
    ax_width, ax_height = bbox.width, bbox.height
    y_diff = np.max(batt_df[y_axis]) - np.min(batt_df[y_axis])
    x_diff = np.max(batt_df[x_axis]) - np.min(batt_df[x_axis])
    yscale = y_diff/ax_height
    xscale = x_diff/ax_width

    ax.plot(batt_df[x_axis], batt_df[y_axis], label=y_axis)
    #filtered = powe_df[powe_df['State'].str.contains('turn')]

    states = set(state_df["State"])
    c_dict = {s : c for s,c in zip(states, sns.color_palette("dark", len(states)))}

    last_px = np.min(batt_df[x_axis])
    last_py = np.min(batt_df[y_axis])
    for index, row in state_df.iterrows():
        text = row["State"] if full_string else re.search(r"[: ]\w+.*", row["State"]).group(0)[1:]
        py = (np.interp(row[x_axis], batt_df[x_axis] , batt_df[y_axis]))
        py2 = np.max(batt_df[y_axis]) + yscale*30 
        px = row[x_axis]

        t = plt.text(x = px - xscale*5 , y = py2, s = text , color ='white', 
                    rotation=0, ha='right', va='bottom', fontsize=12,
                    bbox=dict(facecolor=c_dict[row["State"]], boxstyle='round'))
        plt.gcf().canvas.draw()
        bbox = t.get_window_extent()
        width, height = xscale*bbox.width, yscale*bbox.height
        if px - width <= last_px:
            py2 = max(py2, last_py+yscale*22)
            t.set_position((px - xscale*5, py2))

        line = lines.Line2D([px, px], [py, py2], lw=1., color=c_dict[row["State"]], clip_on=False)
        ax.add_line(line)
        last_py = py2
        last_px = px

    if twin_ax:
        ax2 = ax.twinx()
        ax2.plot(batt_df[x_axis], batt_df[y_axis2], color='coral', label=y_axis2)
        ax2.set_ylabel(y_axis2)

    ax.set_ylabel(y_axis)
    ax.set_xlabel("Time (Minutes)")
    plt.show()
    #fig.legend(loc="lower left", bbox_to_anchor=(0,0), bbox_transform=ax.transAxes)

#power_profile(batt_df, powe_df, "Minutes", "Power")   

## Plotting
**start_index:**
The index in the list of states where the plot starts <br>
The list of choices you have for this is based on where 'beginning_state_string' is found in the log <br>

**end_index:**
The index in the list of states where the plot ends <br>
The list of choices you have for this is based on where 'end_state_string' is found in the log and the max index is added <br>

**full_string:** 
Set this to true to see the full state string <br>
This is useful if you want to change the beginning or end state string or if you need more detail<br>

Press Run Interact to plot graph make sure start_index < end_index. <br>
Can take a fairly long time to run if you're plotting a lot of data at once since there will be more state string boxes trying to be drawn in the same horizontal space causing the image to scale vertically possibly quite a bit.

In [ ]:
interact_manual(power_profile, 
         batt_df=fixed(batt_df), 
         state_df=fixed(powe_df),
         x_axis=fixed("Minutes"),
         y_axis=['Power', 'Temperature', 'Voltage', 'Cap', 'Current'], 
         y_axis2=[None, 'Power', 'Temperature', 'Voltage', 'Cap', 'Current'],
         start_index=starts,
         end_index=ends,
         full_string=[False, True])